In [1]:
import json
import time
import pandas as pd

from factorio_rcon import RCONClient

rcon_client = RCONClient("localhost", 27100, "factorio")

In [2]:
def pause():
    res = rcon_client.send_command("/c rcon.print(game.tick_paused); game.tick_paused=true; rcon.print(game.tick_paused)")
    print(res)
    
def resume():
    res = rcon_client.send_command("/c rcon.print(game.tick_paused); game.tick_paused=false; rcon.print(game.tick_paused)")
    print(res)
    
interfaces = json.loads(rcon_client.send_command("/c rcon.print(helpers.table_to_json(remote.interfaces))"))
interfaces

{'agent': {'create_agents': True,
  'destroy_agents': True,
  'update_agent_friends': True,
  'update_agent_enemies': True,
  'list_agent_forces': True},
 'entities': {'set_recipe': True,
  'set_filter': True,
  'set_inventory_limit': True,
  'remove_inventory_item': True,
  'set_inventory_item': True,
  'extract_inventory_items': True,
  'rotate': True},
 'factorio_verse_docs': {'get_all_methods': True, 'get_method_schema': True},
 'map': {'get_charted_chunks': True,
  'get_map_area_state': True,
  'set_map_area_state': True,
  'clear_map_area': True,
  'get_chunk_lookup': True},
 'research': {'reset_research': True}}

In [4]:
rcon_client.send_command("/c local res = remote.call('agent', 'create_agents', 1, true)")

interfaces = json.loads(rcon_client.send_command("/c rcon.print(helpers.table_to_json(remote.interfaces))"))
interfaces["agent_1"]

{'walk_to': True,
 'stop_walking': True,
 'mine_resource': True,
 'stop_mining': True,
 'craft_enqueue': True,
 'craft_dequeue': True,
 'set_entity_recipe': True,
 'set_entity_filter': True,
 'set_inventory_limit': True,
 'take_inventory_item': True,
 'put_inventory_item': True,
 'place_entity': True,
 'pickup_entity': True,
 'remove_ghost': True,
 'teleport': True,
 'inspect': True,
 'get_placement_cues': True,
 'get_chunks_in_view': True,
 'get_recipes': True,
 'get_technologies': True,
 'get_activity_state': True,
 'enqueue_research': True,
 'cancel_current_research': True,
 'get_reachable': True,
 'get_reachable_full': True,
 'inspect_state': True}

In [4]:
interfaces

{'agent': {'create_agents': True,
  'destroy_agents': True,
  'update_agent_friends': True,
  'update_agent_enemies': True,
  'list_agent_forces': True},
 'entities': {'set_recipe': True,
  'set_filter': True,
  'set_inventory_limit': True,
  'remove_inventory_item': True,
  'set_inventory_item': True,
  'extract_inventory_items': True,
  'rotate': True},
 'factorio_verse_docs': {'get_all_methods': True, 'get_method_schema': True},
 'map': {'get_charted_chunks': True,
  'get_map_area_state': True,
  'set_map_area_state': True,
  'clear_map_area': True,
  'get_chunk_lookup': True},
 'research': {'reset_research': True}}

In [20]:
res = rcon_client.send_command("""/c 
local surface = game.surfaces[1]
local entity_name="stone-furnace"
local position={-12,-7}
local direction = defines.direction.west
    
local can_place_params = {
    name = entity_name,
    position = position,
    direction = direction,
    force = storage.agents[1].character.force,
    build_check_type = defines.build_check_type.manual_ghost,
}
if game.surfaces[1].can_place_entity(can_place_params) then
    surface.create_entity({
        name="entity-ghost",
        inner_name=entity_name,
        position=position,
        direction=direction,
        force=storage.agents[1].character.force
    })
end
""")

res

In [5]:
res = rcon_client.send_command("""/c 
local inventory = storage.agents[1].character.get_main_inventory()
inventory.insert({name="stone-furnace", count=3})
""")

res

In [8]:
res = rcon_client.send_command("""/c 
local entity_name="stone-furnace"
local position={x=-10,y=-10}
-- local direction = defines.direction.west

res = remote.call("agent_1", "place_entity", entity_name, position, nil, true)
rcon.print(helpers.table_to_json(res))
""")

json.loads(res)

{'success': True,
 'position': {'x': -10, 'y': -10},
 'entity_name': 'stone-furnace',
 'entity_type': 'entity-ghost'}

In [9]:
res = rcon_client.send_command("""/c 
local entity_name="stone-furnace"
local position={x=-10,y=-10}
-- local direction = defines.direction.west

res = remote.call("agent_1", "remove_ghost", entity_name, position, nil, true)
rcon.print(helpers.table_to_json(res))
""")

json.loads(res)

{'success': True,
 'entity_name': 'stone-furnace',
 'position': {'x': -10, 'y': -10}}

In [83]:
res = rcon_client.send_command("""/c 

-- Define the area you want to search
local search_area = {
    {x = -12, y = -12},
    {x = -8, y = -8}
}

-- Use find_entities_filtered with the type set to "entity-ghost"
local ghosts = game.surfaces[1].find_entities_filtered({
    position = {-10,-10},
    type = "entity-ghost",
    --name = "stone-furnace"
})

game.print(#(ghosts))

game.print(ghosts[1].ghost_name)
""")
res

In [49]:
res = rcon_client.send_command("""/sc 
local res = game.surfaces[1].find_entities_filtered({position = {x=5.8, y=-103.2}, radius=3, type="tree"})
local buffer = game.create_inventory(20)
if res then
    rcon.print(res[1].type)
    local success = res[1].mine({inventory = buffer})
    if success then
        rcon.print(helpers.table_to_json(buffer.get_contents()))
    end
    buffer.destroy()
end
""")

res

'Cannot execute command. Error: [string "..."]:5: attempt to index field \'?\' (a nil value)'

In [4]:
chunk_lookup = json.loads(rcon_client.send_command("""/c 
local res = remote.call('map', 'get_chunk_lookup', 1, true);
rcon.print(helpers.table_to_json(res));
"""))
chunk_lookup = pd.DataFrame(chunk_lookup).T
chunk_lookup.head()

,resource,entities,water,dirty,snapshot_tick
"-2,-2",{},{},False,False,1711715
"-2,-1",{},{'rocks': True},False,False,1711716
"-2,0",{},{'trees': True},True,False,1711717
"-2,1",{},{'trees': True},True,False,1711718
"-2,2",{},{'trees': True},True,False,1711719


In [42]:
import pandas as pd
chunk_lookup = json.loads(rcon_client.send_command("""/c 
local res = remote.call('map', 'get_chunk_lookup', 1, true);
rcon.print(helpers.table_to_json(res));
"""))
chunk_lookup = pd.DataFrame(chunk_lookup).T

chunks_in_view = json.loads(rcon_client.send_command("""/c 
local res = remote.call('agent_1', 'get_chunks_in_view', 'offshore-pump');
rcon.print(helpers.table_to_json(res));
"""))

relavant_chunks = chunk_lookup[chunk_lookup.index.isin([f"{v['x']},{v['y']}" for v in chunks_in_view])&chunk_lookup.water].index
relavant_chunks = [k.split(",") for k in relavant_chunks]
relavant_chunks = [{'x':int(k[0]), 'y':int(k[1])} for k in relavant_chunks]

for chunk_id in relavant_chunks:
    br = chunk_id['x']*32+31, chunk_id['y']*32+31
    tl = chunk_id['x']*32, chunk_id['y']*32
    print(f"TL: {tl}, BR: {br}")

TL: (-64, 0), BR: (-33, 31)
TL: (-64, 32), BR: (-33, 63)
TL: (-64, 64), BR: (-33, 95)
TL: (-32, 32), BR: (-1, 63)


In [11]:
rcon_client.send_command("""/c 

rendering.draw_text({text="stone +1", surface=game.surfaces[1], color='white', target=storage.agents[1].character, time_to_live=120})

""")

In [5]:
can_place = json.loads(rcon_client.send_command("""/c 
local res = remote.call('agent_1', 'get_placement_cues', 'offshore-pump');
rcon.print(helpers.table_to_json(res));
"""))
can_place

[{'position': {'x': -63, 'y': 30}, 'can_place': True, 'direction': 'south'},
 {'position': {'x': -61, 'y': 29}, 'can_place': True, 'direction': 'south'},
 {'position': {'x': -59, 'y': 30}, 'can_place': True, 'direction': 'south'},
 {'position': {'x': -57, 'y': 31}, 'can_place': True, 'direction': 'south'},
 {'position': {'x': -55, 'y': 32}, 'can_place': True, 'direction': 'south'},
 {'position': {'x': -53, 'y': 33}, 'can_place': True, 'direction': 'south'},
 {'position': {'x': -48, 'y': 35}, 'can_place': True, 'direction': 'south'},
 {'position': {'x': -47, 'y': 35}, 'can_place': True, 'direction': 'south'},
 {'position': {'x': -46, 'y': 35}, 'can_place': True, 'direction': 'south'},
 {'position': {'x': -45, 'y': 35}, 'can_place': True, 'direction': 'south'},
 {'position': {'x': -43, 'y': 36}, 'can_place': True, 'direction': 'south'},
 {'position': {'x': -41, 'y': 35}, 'can_place': True, 'direction': 'south'},
 {'position': {'x': -39, 'y': 34}, 'can_place': True, 'direction': 'south'},

In [49]:
# rcon_client.send_command("/c r")
res = rcon_client.send_command("""/c 
local res = game.players[1].force.get_item_production_statistics(game.surfaces[1]);
rcon.print(helpers.table_to_json({input = res.input_counts, output = res.output_counts}));
-- rcon.print()
""")
json.loads(res if res else "{}")

{'input': {'iron-chest': 76,
  'steel-chest': 29,
  'storage-tank': 14,
  'transport-belt': 24168,
  'fast-transport-belt': 18,
  'underground-belt': 1808,
  'splitter': 150,
  'inserter': 12302,
  'long-handed-inserter': 1969,
  'fast-inserter': 813,
  'small-electric-pole': 2282,
  'pipe': 20276,
  'pipe-to-ground': 180,
  'rail': 18230,
  'construction-robot': 600,
  'passive-provider-chest': 28,
  'storage-chest': 1,
  'roboport': 21,
  'decider-combinator': 100,
  'stone-brick': 23435,
  'concrete': 1000,
  'boiler': 170,
  'steam-engine': 437,
  'burner-mining-drill': 36,
  'electric-mining-drill': 1351,
  'offshore-pump': 11,
  'pumpjack': 8,
  'stone-furnace': 317,
  'steel-furnace': 949,
  'electric-furnace': 625,
  'assembling-machine-1': 994,
  'assembling-machine-2': 965,
  'oil-refinery': 52,
  'chemical-plant': 61,
  'lab': 95,
  'beacon': 28,
  'speed-module': 2412,
  'productivity-module': 1035,
  'productivity-module-2': 20,
  'productivity-module-3': 4,
  'rocket-silo

In [43]:
# rcon_client.send_command("/c r")
res = rcon_client.send_command("""/c 
local surface = game.surfaces[1]
-- local pole = surface.find_entities_filtered({area = {{-10, -10}, {10, 10}}, type = "electric-pole"})[1]
-- local stats = pole.electric_network_statistics;
surface.create_global_electric_network()
local stats = surface.global_electric_network_statistics;
rcon.print(surface.has_global_electric_network)
-- local stats = surface.pollution_statistics
rcon.print(helpers.table_to_json({input = stats.input_counts, output = stats.output_counts}));
-- rcon.print(pole.position)
""")
# json.loads(res if res else "{}")
res

'true\n{"input":{"inserter":25258419168.641803741455078125,"long-handed-inserter":10372725345.687408447265625,"fast-inserter":20417708985.62395477294921875,"roboport":23264501646.6495361328125,"decider-combinator":118131000.01485443115234375,"electric-mining-drill":282472076674.6640625,"pumpjack":1847538230.83154296875,"assembling-machine-1":7695465120.136322021484375,"assembling-machine-2":230978619489.17999267578125,"oil-refinery":56924553368.916412353515625,"chemical-plant":20648897096.70672607421875,"lab":14275448062.597614288330078125,"beacon":13094274133.21875,"rocket-silo":1337734987.51800537109375,"radar":4391326414.4091796875},"output":{"steam-engine":713097418087.8272705078125}}'

In [69]:
recipes = json.loads(rcon_client.send_command("""/c 
local res = remote.call('agent_1', 'get_recipes');
rcon.print(helpers.table_to_json(res));
"""))
recipes

[{'name': 'wooden-chest',
  'category': 'crafting',
  'energy': 0.5,
  'ingredients': [{'type': 'item', 'name': 'wood', 'amount': 2}]},
 {'name': 'iron-chest',
  'category': 'crafting',
  'energy': 0.5,
  'ingredients': [{'type': 'item', 'name': 'iron-plate', 'amount': 8}]},
 {'name': 'transport-belt',
  'category': 'crafting',
  'energy': 0.5,
  'ingredients': [{'type': 'item', 'name': 'iron-plate', 'amount': 1},
   {'type': 'item', 'name': 'iron-gear-wheel', 'amount': 1}]},
 {'name': 'burner-inserter',
  'category': 'crafting',
  'energy': 0.5,
  'ingredients': [{'type': 'item', 'name': 'iron-plate', 'amount': 1},
   {'type': 'item', 'name': 'iron-gear-wheel', 'amount': 1}]},
 {'name': 'stone-brick',
  'category': 'smelting',
  'energy': 3.2,
  'ingredients': [{'type': 'item', 'name': 'stone', 'amount': 2}]},
 {'name': 'burner-mining-drill',
  'category': 'crafting',
  'energy': 2,
  'ingredients': [{'type': 'item', 'name': 'iron-plate', 'amount': 3},
   {'type': 'item', 'name': 'iro

In [90]:
technologies = json.loads(rcon_client.send_command("""/c 
local res = remote.call('agent_1', 'get_technologies');
rcon.print(helpers.table_to_json(res));
"""))
technologies
tech_df = pd.DataFrame(technologies)
level = {0: {"nil": tech_df[tech_df.prerequisites.apply(lambda x: not len(x))].name.tolist()}}
k = 0
while(True):
    try:
        l = {}
        for name in level[k]:
            for dep in level[k][name]:
                l[dep] = tech_df[tech_df.prerequisites.apply(lambda x: dep in x)].name.tolist()
        level[k+1] = l
        if not len(level[k+1]):
            break
        k+=1
    except:
        break

,name,researched,enabled,prerequisites,successors,research_unit_ingredients,research_unit_count,research_unit_energy,saved_progress,effects,research_trigger
13,automation-science-pack,False,True,"{'steam-power': None, 'electronics': None}","{'automation': None, 'electric-mining-drill': ...",{},1,0,0,"[{'type': 'unlock-recipe', 'recipe': 'automati...","{'type': 'craft-item', 'item': {'name': 'lab'}..."
46,electronics,False,True,{},{'automation-science-pack': None},{},1,0,0,"[{'type': 'unlock-recipe', 'recipe': 'copper-c...","{'type': 'craft-item', 'item': {'name': 'coppe..."
117,oil-processing,False,True,{'oil-gathering': None},"{'flammables': None, 'plastics': None, 'sulfur...",{},1,0,0,"[{'type': 'unlock-recipe', 'recipe': 'oil-refi...","{'type': 'mine-entity', 'entity': 'crude-oil'}"
158,space-science-pack,False,True,{'rocket-silo': None},"{'artillery-shell-range-1': None, 'artillery-s...",{},1,0,0,{},"{'type': 'send-item-to-orbit', 'item': {'name'..."
163,steam-power,False,True,{},{'automation-science-pack': None},{},1,0,0,"[{'type': 'unlock-recipe', 'recipe': 'pipe'}, ...","{'type': 'craft-item', 'item': {'name': 'iron-..."
164,steel-axe,False,True,{'steel-processing': None},{},{},1,0,0,"[{'type': 'character-mining-speed', 'modifier'...","{'type': 'craft-item', 'item': {'name': 'steel..."
179,uranium-processing,False,True,{'uranium-mining': None},"{'kovarex-enrichment-process': None, 'nuclear-...",{},1,0,0,"[{'type': 'unlock-recipe', 'recipe': 'centrifu...","{'type': 'mine-entity', 'entity': 'uranium-ore'}"


In [4]:
res = json.loads(rcon_client.send_command("""
/c local success, result = xpcall(function() 
    return remote.call('agent_1', 'walk_to', {x=25,y=-85}) 
end, debug.traceback)
if success then 
    rcon.print(helpers.table_to_json(result)) 
else 
    rcon.print(helpers.table_to_json({success = false, error = result})) 
end
"""))

if not res["success"]:
    print(res["error"])
else:
    print(res)

{'success': True, 'queued': True, 'action_id': 'walk_to_3474361_1', 'tick': 3474361, 'result': {'path_id': 1, 'options_used': {'goal': {'x': 25, 'y': -85}, 'start': {'y': -2, 'x': -2}, 'bounding_box': {'left_top': {'y': -0.19921875, 'x': -0.19921875}, 'right_bottom': {'y': 0.19921875, 'x': 0.19921875}}, 'collision_mask': {'layers': {'is_object': True, 'player': True, 'train': True}, 'consider_tile_transitions': True}, 'force': 'player', 'radius': 0, 'entity_to_ignore': 'character_Agent-1'}}}


In [23]:
rcon_client.send_command("/c rcon.print(helpers.table_to_json(remote.call('agent_1', 'stop_walking')))")

'{"success":true,"position":{"y":-32.48046875,"x":-38.62890625}}'

In [137]:
rcon_client.send_command("""
/c local success, result = xpcall(function() 
    return remote.call('agent_1', 'inspect', false, true) 
end, debug.traceback)
if success then 
    rcon.print(helpers.table_to_json({success = true, result = result})) 
else 
    rcon.print(helpers.table_to_json({success = false, error = result})) 
end
""")

'{"success":true,"result":{"agent_id":1,"tick":1586673,"position":{"x":0,"y":0},"reachable_resources":{},"reachable_entities":{}}}'

In [14]:
res = rcon_client.send_command("/c rcon.print(helpers.table_to_json(remote.call('agent_1', 'inspect', true, false)));")
print(json.loads(res))

res = json.loads(rcon_client.send_command("""
/c local success, result = xpcall(function() 
    return remote.call('agent_1', 'mine_resource', 'rock', 50) 
end, debug.traceback)
if success then 
    rcon.print(helpers.table_to_json({success = true, result = result})) 
else 
    rcon.print(helpers.table_to_json({success = false, error = result})) 
end
"""))

if not res["success"]:
    print(res["error"])
else:
    print(res)

{'agent_id': 1, 'tick': 24072, 'position': {'x': 53.328125, 'y': -17.3984375}, 'inventory': {}}
{'success': True, 'result': {'success': True, 'queued': True, 'action_id': 'mine_resource_24073_1', 'tick': 24073, 'estimated_ticks': 240, 'products': [{'type': 'item', 'name': 'stone', 'probability': 1, 'amount': 20}]}}


In [47]:
res = rcon_client.send_command("/c rcon.print(helpers.table_to_json(remote.call('agent_1', 'inspect', true, false)));")
print(json.loads(res))

{'agent_id': 1, 'tick': 967619, 'position': {'x': -65.80078125, 'y': 43.9453125}, 'inventory': [{'name': 'coal', 'quality': 'normal', 'count': 20}, {'name': 'stone', 'quality': 'normal', 'count': 30}]}


In [53]:
res = rcon_client.send_command("/c rcon.print(helpers.table_to_json(remote.call('agent_1', 'inspect', true, false)));")
print(json.loads(res))

res = json.loads(rcon_client.send_command("""
/c local success, result = xpcall(function() 
    return remote.call('agent_1', 'craft_enqueue', 'stone-furnace', 2) 
end, debug.traceback)
if success then 
    rcon.print(helpers.table_to_json({success = true, result = result})) 
else 
    rcon.print(helpers.table_to_json({success = false, error = result})) 
end
"""))

if not res["success"]:
    print(res["error"])
else:
    print(res)

{'agent_id': 1, 'tick': 971497, 'position': {'x': -65.80078125, 'y': 43.9453125}, 'inventory': [{'name': 'stone-furnace', 'quality': 'normal', 'count': 6}, {'name': 'coal', 'quality': 'normal', 'count': 20}]}
Error when running interface function agent_1.craft_enqueue: __level__/types/agent/crafting.lua:82: Agent: Cannot craft recipe: insufficient ingredients or recipe not available
stack traceback:
	[C]: in function 'error'
	__level__/types/agent/crafting.lua:82: in function <__level__/types/agent/crafting.lua:51>
	(...tail calls...)
stack traceback:
	[C]: in function 'call'
	[string "local success, result = xpcall(function() ..."]:2: in function <[string "local success, result = xpcall(function() ..."]:1>
	[C]: in function 'xpcall'
	[string "local success, result = xpcall(function() ..."]:1: in main chunk


In [54]:
res = rcon_client.send_command("/c rcon.print(helpers.table_to_json(remote.call('agent_1', 'inspect', true, false)));")
print(json.loads(res))


{'agent_id': 1, 'tick': 971757, 'position': {'x': -65.80078125, 'y': 43.9453125}, 'inventory': [{'name': 'stone-furnace', 'quality': 'normal', 'count': 6}, {'name': 'coal', 'quality': 'normal', 'count': 20}]}


In [11]:
import pandas as pd
import numpy as np

def nearest_tile(df, x, y, resource_name, exclude_self=True):
    sub = df[df["resource_name"] == resource_name]
    if sub.empty:
        return None  # or raise

    dx = sub["x"].to_numpy() - x
    dy = sub["y"].to_numpy() - y
    d2 = dx*dx + dy*dy  # squared Euclidean distance

    if exclude_self:
        mask = (dx != 0) | (dy != 0)
        if not mask.any():
            return None
        d2 = np.where(mask, d2, np.inf)

    i = d2.argmin()
    row = sub.iloc[i][["patch_id", "x", "y", "resource_name"]]
    return row.to_dict()       # the nearest row

resources = json.loads(rcon_client.send_command("/c remote.call('helpers', 'take_resources')"))
print(resources)
with open(f".fv-output/output_0/script-output/factoryverse/resources.snap-{resources['tick']}.json", "r") as f:
    data = json.load(f)

df_r = pd.DataFrame(data["data"]["rows"])
# df_r

origin = {"x": 0, "y":0}

# iron_tile = nearest_tile(df_r, origin["x"], origin["y"], "iron-ore")
stone_tile = nearest_tile(df_r, iron_tile["x"], iron_tile["y"], "stone")
coal_tile = nearest_tile(df_r, origin["x"], origin["y"], "coal")
iron_tile

{'surface': 'nauvis', 'patch_count': 9, 'row_count': 173, 'tick': 185265}


{'patch_id': 'iron-ore;-14;-64',
 'x': -26,
 'y': -51,
 'resource_name': 'iron-ore'}

In [31]:
resource = iron_tile
resource = coal_tile
resource = stone_tile

args = f"""{{
    agent_id = {1},
    x = {resource['x']},
    y = {resource['y']},
    resource_name = '{resource['resource_name']}',
    max_count = {30},
    walk_if_unreachable = {str(True).lower()},
}}"""

rcon_client.send_command(f"/c remote.call('actions', 'mine_resource', {args})")
print(f"/c remote.call('actions', 'mine_resource', {args})")

/c remote.call('actions', 'mine_resource', {
    agent_id = 1,
    x = 55,
    y = -63,
    resource_name = 'stone',
    max_count = 30,
    walk_if_unreachable = true,
})


In [58]:
rcon_client.send_command("/c remote.call('helpers', 'reset_agents_state')")

'[helpers.reset_agents_state] Stopped all agent walking/mining and cleared pending jobs/intents.'

In [50]:
json.loads(rcon_client.send_command("/c remote.call('helpers', 'print_agent_inventory', 1)"))

{'inventory': [{'name': 'stone-furnace', 'count': 6}]}

In [33]:
rcon_client.send_command("/c game.speed=1.0")


In [40]:
rcon_client.send_command(f"/c remote.call('actions', 'item_craft', {{agent_id=1, recipe='stone-furnace', count = 5}})")


In [55]:
rcon_client.send_command("/c remote.call('actions', 'entity_place', {agent_id=1, entity_name='stone-furnace', position = {x=54, y=-58}})")



In [108]:
c = """
local agent = storage.agent_characters[1];
local y = agent.position.y;
local x = agent.position.x;
local chunk_x = math.floor(x / 32);
local chunk_y = math.floor(y / 32);

rcon.print(game.forces["player"].is_chunk_charted(game.surfaces[1], {chunk_x,chunk_y}))

rcon.print(tostring(chunk_x) .. ", " .. tostring(chunk_y));

function get_chunk_bbox(chunk_x, chunk_y)
    local chunk_size = 32
    local left_top_x = chunk_x * chunk_size
    local left_top_y = chunk_y * chunk_size
    return {
        left_top = {x = left_top_x, y = left_top_y},
        right_bottom = {x = left_top_x + chunk_size - 1, y = left_top_y + chunk_size - 1}
    }
end

game.forces["player"].chart(game.surfaces[1], {chunk_x,chunk_y })
"""

print(rcon_client.send_command(f"/c {c}"))

true
0, 0


In [25]:
# for i in range(1, 21):
#     rcon_client.send_command(f"""/c remote.call('actions', "agent.walk_to", {{
#       agent_id = {i},
#       goal = {{ x = {i-10 if i%4==0 else -i-10}, y = {-i if i%4==0 else i} }},
#       -- Manhattan-leaning defaults:
#       prefer_cardinal = true,
#       diag_band = 0.25,
#       snap_axis_eps = 0.25,
#       arrive_radius = 0.7,
#       replan_on_stuck = true,
#     }})""")

    
rcon_client.send_command("""/c remote.call('actions',"agent.walk_to", {
  agent_id = 1,
  -- Manhattan-leaning defaults:
  goal = { x = 123, y = 300 },
  prefer_cardinal = true,
  diag_band = 0.25,
  snap_axis_eps = 0.25,
  arrive_radius = 0.7,
  replan_on_stuck = true,
})""")

In [60]:
oil = json.loads(rcon_client.send_command("/c remote.call('helpers', 'take_crude_oil')"))
oil

{'surface': 'nauvis',
 'crude': {'well_count': 130, 'patch_count': 14, 'total_amount': 523955402},
 'tick': 377153}

In [8]:
entities = json.loads(rcon_client.send_command("/c remote.call('helpers', 'take_entities')"))
entities

{'surface': 'nauvis',
 'tick': 377153,
 'entities': {'entity_rows': 12100,
  'electric_rows': 12100,
  'crafting_rows': 1893,
  'burner_rows': 1117,
  'inventory_rows': 3577,
  'fluids_rows': 2384,
  'inserter_rows': 4432}}

In [6]:
belts = json.loads(rcon_client.send_command("/c remote.call('helpers', 'take_belts')"))
belts

{'surface': 'nauvis', 'belts': {'total': 10802}, 'tick': 377153}

In [120]:
waters = json.loads(rcon_client.send_command("/c remote.call('helpers', 'take_water')"))
waters

{'surface': 'nauvis',
 'water': {'patch_count': 1,
  'top': [{'tiles': 2380,
    'bbox': {'min_x': -60, 'min_y': 34, 'max_x': -3, 'max_y': 99},
    'centroid': {'x': -30.26953125, 'y': 66.4140625}}]},
 'tick': 438879}

In [154]:
# iron_tile["x"]=-16.5
# iron_tile["y"]=-64.5

/c remote.call('actions', 'mine_resource', {
    agent_id = 1,
    x = 36.5,
    y = -85.5,
    resource_name = 'coal',
    max_count = 10,
    walk_if_unreachable = true,
})


'[helpers.reset_agents_state] Stopped all agent walking/mining and cleared pending jobs/intents.'

{'inventory': [{'name': 'iron-ore', 'count': 50},
  {'name': 'iron-ore', 'count': 10}]}

In [176]:
df_water

,tick,patch_id,tiles,bbox_min_x,bbox_min_y,bbox_max_x,bbox_max_y,centroid_x,centroid_y,sum_x,sum_y
0,44596,1,264,-21,-3,-11,20,-16.000000,8.500000,-4224,2244
1,44596,2,253,-21,70,-11,94,-15.921875,81.183594,-4028,20539


In [149]:
df_water_spans.iloc[df_water_spans.tile_count.argmax()]

tick          264252
patch_id           1
row_y             62
x                -58
len               56
tile_count        56
Name: 28, dtype: int64

In [151]:
df_water_spans[df_water_spans.x==-46]

,tick,patch_id,row_y,x,len,tile_count
15,264252,1,49,-46,33,33
16,264252,1,50,-46,33,33
55,264252,1,88,-46,19,19


In [153]:
df_water_spans[df_water_spans.x==-23]

,tick,patch_id,row_y,x,len,tile_count
56,264252,1,88,-23,11,11


In [154]:
df_water_spans.value_counts("row_y")

row_y
87    2
88    2
89    2
34    1
82    1
     ..
61    1
62    1
63    1
64    1
99    1
Name: count, Length: 66, dtype: int64

In [65]:
with open(".fv-output/output_0/script-output/factoryverse/entities.snap-377153.json", "r") as f:
    data = json.load(f)

In [83]:
# columns = set()
from collections import defaultdict
cols = defaultdict(set)
entities = []

for chunk in data["data"]["chunks"]:
    for entity in chunk["entities"]:
        entities.append(entity)
        cols[entity["name"]] = cols[entity["name"]].union(list(entity.keys()))

In [81]:
[(k, len(v))  for k, v in cols.items()]

[('pumpjack', 18),
 ('small-electric-pole', 18),
 ('pipe', 18),
 ('steam-engine', 18),
 ('pipe-to-ground', 18),
 ('offshore-pump', 18),
 ('boiler', 18),
 ('inserter', 18),
 ('electric-mining-drill', 18),
 ('assembling-machine-2', 18),
 ('long-handed-inserter', 18),
 ('chemical-plant', 18),
 ('roboport', 18),
 ('radar', 18),
 ('passive-provider-chest', 18),
 ('stone-furnace', 18),
 ('storage-tank', 18),
 ('oil-refinery', 18),
 ('fast-inserter', 18),
 ('steel-furnace', 18),
 ('iron-chest', 18),
 ('burner-mining-drill', 18),
 ('assembling-machine-1', 18),
 ('lab', 18),
 ('storage-chest', 18),
 ('beacon', 18),
 ('rocket-silo', 18),
 ('decider-combinator', 18)]

In [84]:
import pandas as pd

df_e = pd.DataFrame(entities)

In [85]:
df_e

,unit_number,name,type,force,position,direction,direction_name,orientation,orientation_name,health,...,energy,electric_buffer_size,inventories,fluids,bounding_box,selection_box,burner,inserter,recipe,crafting_progress
0,6381,pumpjack,mining-drill,player,"{'y': 170.5, 'x': -164.5}",8,south,0.50,south,200,...,1600.0,1600.0,{'mining_drill_modules': {}},"[{'index': 1, 'name': 'crude-oil', 'amount': 5...","{'min_x': -165.69921875, 'min_y': 169.30078125...","{'min_x': -166, 'min_y': 169, 'max_x': -163, '...",NaN,NaN,NaN,NaN
1,6379,small-electric-pole,electric-pole,player,"{'y': 170.5, 'x': -161.5}",0,north,0.00,north,100,...,0.0,NaN,NaN,NaN,"{'min_x': -161.6484375, 'min_y': 170.3515625, ...","{'min_x': -161.8984375, 'min_y': 170.1015625, ...",NaN,NaN,NaN,NaN
2,6385,pipe,pipe,player,"{'y': 172.5, 'x': -164.5}",0,north,0.00,north,100,...,0.0,NaN,NaN,"[{'index': 1, 'name': 'crude-oil', 'amount': 9...","{'min_x': -164.98828125, 'min_y': 172.2109375,...","{'min_x': -165, 'min_y': 172, 'max_x': -164, '...",NaN,NaN,NaN,NaN
3,6384,pipe,pipe,player,"{'y': 172.5, 'x': -165.5}",0,north,0.00,north,100,...,0.0,NaN,NaN,"[{'index': 1, 'name': 'crude-oil', 'amount': 9...","{'min_x': -165.7890625, 'min_y': 172.01171875,...","{'min_x': -166, 'min_y': 172, 'max_x': -165, '...",NaN,NaN,NaN,NaN
4,6387,pipe,pipe,player,"{'y': 172.5, 'x': -162.5}",0,north,0.00,north,100,...,0.0,NaN,NaN,"[{'index': 1, 'name': 'crude-oil', 'amount': 9...","{'min_x': -162.98828125, 'min_y': 172.2109375,...","{'min_x': -163, 'min_y': 172, 'max_x': -162, '...",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12095,41103,electric-mining-drill,mining-drill,player,"{'y': 281.5, 'x': 198.5}",12,west,0.75,west,300,...,1600.0,1600.0,{'mining_drill_modules': {}},NaN,"{'min_x': 197.15234375, 'min_y': 280.15234375,...","{'min_x': 197, 'min_y': 280, 'max_x': 200, 'ma...",NaN,NaN,NaN,NaN
12096,41277,small-electric-pole,electric-pole,player,"{'y': 283.5, 'x': 196.5}",0,north,0.00,north,100,...,0.0,NaN,NaN,NaN,"{'min_x': 196.3515625, 'min_y': 283.3515625, '...","{'min_x': 196.1015625, 'min_y': 283.1015625, '...",NaN,NaN,NaN,NaN
12097,41122,electric-mining-drill,mining-drill,player,"{'y': 284.5, 'x': 191.5}",12,west,0.75,west,300,...,1600.0,1600.0,{'mining_drill_modules': {}},NaN,"{'min_x': 190.15234375, 'min_y': 283.15234375,...","{'min_x': 190, 'min_y': 283, 'max_x': 193, 'ma...",NaN,NaN,NaN,NaN
12098,41125,electric-mining-drill,mining-drill,player,"{'y': 284.5, 'x': 194.5}",4,east,0.25,east,300,...,1600.0,1600.0,{'mining_drill_modules': {}},NaN,"{'min_x': 193.15234375, 'min_y': 283.15234375,...","{'min_x': 193, 'min_y': 283, 'max_x': 196, 'ma...",NaN,NaN,NaN,NaN


In [90]:
result = (
    df_e.groupby("name")
        .apply(lambda g: g.notna().any().loc[lambda s: s].index.tolist())
)


/var/folders/9c/w40jfx7n2jv7thv2nk09p1sc0000gn/T/ipykernel_85090/3103080504.py:3: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g.notna().any().loc[lambda s: s].index.tolist())


   name                  
0  assembling-machine-1      [unit_number, name, type, force, position, dir...
   assembling-machine-2      [unit_number, name, type, force, position, dir...
   beacon                    [unit_number, name, type, force, position, dir...
   boiler                    [unit_number, name, type, force, position, dir...
   burner-mining-drill       [unit_number, name, type, force, position, dir...
   chemical-plant            [unit_number, name, type, force, position, dir...
   decider-combinator        [unit_number, name, type, force, position, dir...
   electric-mining-drill     [unit_number, name, type, force, position, dir...
   fast-inserter             [unit_number, name, type, force, position, dir...
   inserter                  [unit_number, name, type, force, position, dir...
   iron-chest                [unit_number, name, type, force, position, dir...
   lab                       [unit_number, name, type, force, position, dir...
   long-handed-inserter   

In [94]:
sorted([(k, len(v))  for k, v in result.to_dict().items()], key=lambda x: x[1], reverse=True)

[('assembling-machine-2', 21),
 ('chemical-plant', 21),
 ('oil-refinery', 21),
 ('assembling-machine-1', 20),
 ('rocket-silo', 20),
 ('pumpjack', 19),
 ('steel-furnace', 19),
 ('stone-furnace', 19),
 ('beacon', 18),
 ('boiler', 18),
 ('electric-mining-drill', 18),
 ('fast-inserter', 18),
 ('inserter', 18),
 ('lab', 18),
 ('long-handed-inserter', 18),
 ('roboport', 18),
 ('steam-engine', 18),
 ('burner-mining-drill', 17),
 ('decider-combinator', 17),
 ('radar', 17),
 ('iron-chest', 16),
 ('offshore-pump', 16),
 ('passive-provider-chest', 16),
 ('pipe', 16),
 ('pipe-to-ground', 16),
 ('storage-chest', 16),
 ('storage-tank', 16),
 ('small-electric-pole', 14)]

In [95]:
from collections import defaultdict

# Convert lists to sets for easier comparison
sets = {k: set(v) for k, v in result.to_dict().items()}

# Group by the actual set (not just length)
groups = defaultdict(list)
for name, cols in sets.items():
    groups[frozenset(cols)].append(name)

# Only keep groups that have more than one member
same_column_sets = {tuple(sorted(k)): v for k, v in groups.items() if len(v) > 1}

print(same_column_sets)

{('bounding_box', 'crafting_progress', 'direction', 'direction_name', 'electric_buffer_size', 'electric_network_id', 'energy', 'force', 'health', 'inventories', 'name', 'orientation', 'orientation_name', 'position', 'recipe', 'selection_box', 'status', 'status_name', 'type', 'unit_number'): ['assembling-machine-1', 'rocket-silo'], ('bounding_box', 'crafting_progress', 'direction', 'direction_name', 'electric_buffer_size', 'electric_network_id', 'energy', 'fluids', 'force', 'health', 'inventories', 'name', 'orientation', 'orientation_name', 'position', 'recipe', 'selection_box', 'status', 'status_name', 'type', 'unit_number'): ['assembling-machine-2', 'chemical-plant', 'oil-refinery'], ('bounding_box', 'direction', 'direction_name', 'electric_buffer_size', 'electric_network_id', 'energy', 'force', 'health', 'inventories', 'name', 'orientation', 'orientation_name', 'position', 'selection_box', 'status', 'status_name', 'type', 'unit_number'): ['beacon', 'electric-mining-drill', 'lab', 'ro

In [100]:
from collections import defaultdict

# Turn lists into sets for comparison
sets = {k: set(v) for k, v in result.to_dict().items()}

# Group by exact set of columns
groups = defaultdict(list)
for name, cols in sets.items():
    groups[frozenset(cols)].append(name)

# Split into common vs unique
common_sets = {tuple(sorted(k)): v for k, v in groups.items() if len(v) > 1}
unique_sets = {tuple(sorted(k)): v for k, v in groups.items() if len(v) == 1}

print("=== Common sets (shared by multiple names) ===")
for colset, names in common_sets.items():
    print(colset, "->", names)

print("\n=== Unique sets (non-common, only one name has them) ===")
for colset, names in unique_sets.items():
    print(colset, "->", names)

=== Common sets (shared by multiple names) ===
('bounding_box', 'crafting_progress', 'direction', 'direction_name', 'electric_buffer_size', 'electric_network_id', 'energy', 'force', 'health', 'inventories', 'name', 'orientation', 'orientation_name', 'position', 'recipe', 'selection_box', 'status', 'status_name', 'type', 'unit_number') -> ['assembling-machine-1', 'rocket-silo']
('bounding_box', 'crafting_progress', 'direction', 'direction_name', 'electric_buffer_size', 'electric_network_id', 'energy', 'fluids', 'force', 'health', 'inventories', 'name', 'orientation', 'orientation_name', 'position', 'recipe', 'selection_box', 'status', 'status_name', 'type', 'unit_number') -> ['assembling-machine-2', 'chemical-plant', 'oil-refinery']
('bounding_box', 'direction', 'direction_name', 'electric_buffer_size', 'electric_network_id', 'energy', 'force', 'health', 'inventories', 'name', 'orientation', 'orientation_name', 'position', 'selection_box', 'status', 'status_name', 'type', 'unit_number')

In [101]:
from collections import defaultdict

sets = {k: set(v) for k, v in result.to_dict().items()}

# Group by exact set
groups = defaultdict(list)
for name, cols in sets.items():
    groups[frozenset(cols)].append(name)

# Common vs unique
common_sets = {s: v for s, v in groups.items() if len(v) > 1}
unique_sets = {s: v for s, v in groups.items() if len(v) == 1}

# Compare uniques with others of the same size
uniqueness_report = {}
for colset, names in unique_sets.items():
    size = len(colset)
    peers = [s for s in groups if len(s) == size and s != colset]
    diffs = []
    for peer in peers:
        diffs.append({
            "peer": groups[peer],
            "only_in_this": sorted(colset - peer),
            "missing_in_this": sorted(peer - colset)
        })
    uniqueness_report[names[0]] = {
        "columns": sorted(colset),
        "size": size,
        "comparisons": diffs
    }

# Example print
for name, report in uniqueness_report.items():
    print(f"\n=== {name} ===")
    print(f"Columns ({report['size']}): {report['columns']}")
    for comp in report["comparisons"]:
        print(f" vs {comp['peer']} -> only_in_this: {comp['only_in_this']}, missing_in_this: {comp['missing_in_this']}")


=== boiler ===
Columns (18): ['bounding_box', 'burner', 'direction', 'direction_name', 'energy', 'fluids', 'force', 'health', 'inventories', 'name', 'orientation', 'orientation_name', 'position', 'selection_box', 'status', 'status_name', 'type', 'unit_number']
 vs ['beacon', 'electric-mining-drill', 'lab', 'roboport'] -> only_in_this: ['burner', 'fluids'], missing_in_this: ['electric_buffer_size', 'electric_network_id']
 vs ['fast-inserter', 'inserter', 'long-handed-inserter'] -> only_in_this: ['burner', 'fluids', 'inventories'], missing_in_this: ['electric_buffer_size', 'electric_network_id', 'inserter']
 vs ['steam-engine'] -> only_in_this: ['burner', 'inventories'], missing_in_this: ['electric_buffer_size', 'electric_network_id']

=== burner-mining-drill ===
Columns (17): ['bounding_box', 'burner', 'direction', 'direction_name', 'energy', 'force', 'health', 'inventories', 'name', 'orientation', 'orientation_name', 'position', 'selection_box', 'status', 'status_name', 'type', 'unit_

In [40]:
import random
import numpy as np

def simulate_status_changes(n_entities, change_prob, n_iterations=10000):
    """
    n_entities: total tracked entities
    change_prob: probability entity changes status in 60 ticks
    """
    changes_per_interval = []
    
    for _ in range(n_iterations):
        # Each entity has change_prob of changing
        n_changes = np.random.binomial(n_entities, change_prob)
        changes_per_interval.append(n_changes)
    
    return np.array(changes_per_interval)

# Scenarios
scenarios = {
    "small_stable": (1_000, 0.02),      # 1k entities, 2% change rate
    "small_active": (1_000, 0.15),      # 1k entities, 15% change rate
    
    "medium_stable": (10_000, 0.02),    # 10k entities, 2% change rate
    "medium_active": (10_000, 0.15),    # 10k entities, 15% change rate
    "medium_chaos": (10_000, 0.40),     # 10k entities, 40% change rate
    
    "large_stable": (50_000, 0.02),     # 50k entities, 2% change rate
    "large_active": (50_000, 0.15),     # 50k entities, 15% change rate
    
    "megabase_stable": (100_000, 0.02), # 100k entities, 2% change rate
    "megabase_active": (100_000, 0.10), # 100k entities, 10% change rate
}

BYTES_PER_CHANGE = 11
HEADER_BYTES = 40

for name, (n_entities, change_rate) in scenarios.items():
    changes = simulate_status_changes(n_entities, change_rate)
    
    # Calculate bytes per packet
    bytes_per_packet = (changes * BYTES_PER_CHANGE) + HEADER_BYTES
    
    # Per second (every 60 ticks)
    bytes_per_second = bytes_per_packet
    
    print(f"\n{name.upper()}")
    print(f"  Entities: {n_entities:,}, Change Rate: {change_rate*100}%")
    print(f"  Mean changes/interval: {changes.mean():.0f}")
    print(f"  Std dev: {changes.std():.0f}")
    print(f"  50th percentile: {np.percentile(changes, 50):.0f} changes")
    print(f"  95th percentile: {np.percentile(changes, 95):.0f} changes")
    print(f"  99th percentile: {np.percentile(changes, 99):.0f} changes")
    print(f"  Bandwidth (mean): {bytes_per_second.mean():.0f} B/s ({bytes_per_second.mean()/1024:.2f} KB/s)")
    print(f"  Bandwidth (p95): {np.percentile(bytes_per_second, 95):.0f} B/s ({np.percentile(bytes_per_second, 95)/1024:.2f} KB/s)")
    print(f"  Bandwidth (p99): {np.percentile(bytes_per_second, 99):.0f} B/s ({np.percentile(bytes_per_second, 99)/1024:.2f} KB/s)")


SMALL_STABLE
  Entities: 1,000, Change Rate: 2.0%
  Mean changes/interval: 20
  Std dev: 4
  50th percentile: 20 changes
  95th percentile: 28 changes
  99th percentile: 31 changes
  Bandwidth (mean): 260 B/s (0.25 KB/s)
  Bandwidth (p95): 348 B/s (0.34 KB/s)
  Bandwidth (p99): 381 B/s (0.37 KB/s)

SMALL_ACTIVE
  Entities: 1,000, Change Rate: 15.0%
  Mean changes/interval: 150
  Std dev: 11
  50th percentile: 150 changes
  95th percentile: 169 changes
  99th percentile: 177 changes
  Bandwidth (mean): 1689 B/s (1.65 KB/s)
  Bandwidth (p95): 1899 B/s (1.85 KB/s)
  Bandwidth (p99): 1987 B/s (1.94 KB/s)

MEDIUM_STABLE
  Entities: 10,000, Change Rate: 2.0%
  Mean changes/interval: 200
  Std dev: 14
  50th percentile: 200 changes
  95th percentile: 223 changes
  99th percentile: 233 changes
  Bandwidth (mean): 2239 B/s (2.19 KB/s)
  Bandwidth (p95): 2493 B/s (2.43 KB/s)
  Bandwidth (p99): 2603 B/s (2.54 KB/s)

MEDIUM_ACTIVE
  Entities: 10,000, Change Rate: 15.0%
  Mean changes/interval: 15